In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import time
import warnings

from tqdm import tqdm

# Suppress unnecessary FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
topics = pd.read_csv('/kaggle/input/sampled_train_topics.csv')
qrels = pd.read_csv('/kaggle/input/sampled_train_qrels.csv')

In [4]:
!curl -s "https://get.sdkman.io" | bash && source "$HOME/.sdkman/bin/sdkman-init.sh" && sdk install java 11.0.22-amzn < /dev/null


                                -+syyyyyyys:
                            `/yho:`       -yd.
                         `/yh/`             +m.
                       .oho.                 hy                          .`
                     .sh/`                   :N`                `-/o`  `+dyyo:.
                   .yh:`                     `M-          `-/osysoym  :hs` `-+sys:      hhyssssssssy+
                 .sh:`                       `N:          ms/-``  yy.yh-      -hy.    `.N-````````+N.
               `od/`                         `N-       -/oM-      ddd+`     `sd:     hNNm        -N:
              :do`                           .M.       dMMM-     `ms.      /d+`     `NMMs       `do
            .yy-                             :N`    ```mMMM.      -      -hy.       /MMM:       yh
          `+d+`           `:/oo/`       `-/osyh/ossssssdNMM`           .sh:         yMMN`      /m.
         -dh-           :ymNMMMMy  `-/shmNm-`:N/-.``   `.sN            /N-         `NMMy      .m/
  

In [5]:
pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 k

In [6]:
%env JAVA_HOME=/root/.sdkman/candidates/java/current
import pyterrier as pt
if not pt.started():
    pt.init()

env: JAVA_HOME=/root/.sdkman/candidates/java/current
terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [7]:
dataset = pt.get_dataset('irds:dpr-w100')

In [ ]:
# Remove stopwords
start_time = time.time()

indexer =  pt.IterDictIndexer("./indexes/stopwords")
indexer.setProperty("termpipelines", "Stopwords")
index_ref = indexer.index(dataset.get_corpus_iter(), fields=['text', 'title'])
index_ref = pt.IndexRef.of("./indexes/stopwords/data.properties")
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

end_time = time.time()
print(end_time - start_time)
len((index.getLexicon()))

In [8]:
index_ref = pt.IndexRef.of("./indexes/stopwords/data.properties")
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

15:55:18.889 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
15:55:23.148 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 160.3 MiB of memory would be required.
15:55:23.184 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 470.4 MiB of memory would be required.
Number of documents: 21015324
Number of terms: 4630820
Number of postings: 1030301407
Number of fields: 

In [ ]:
search_times = []

# randomly selected 100 items
ix_range = random.sample(range(len(index.getLexicon())), 100)

# timimg
for ix, kv in enumerate(index.getLexicon()):
    start_time = time.time()
    if ix in ix_range:
        pointer = index.getLexicon()[kv.getKey()]
    end_time = time.time()
    search_time = end_time - start_time
    search_times.append(search_time)
    
print(sum(search_times)/100)

In [9]:
topics["qid"] = topics["qid"].astype(str)
qrels["qid"] = qrels["qid"].astype(str)
qrels["docno"] = qrels["docno"].astype(str)

In [ ]:
from sklearn.model_selection import KFold

# 5-folds
k = 5
topics_shuffled = topics.sample(frac=1, random_state=34)

In [ ]:
type(topics[0:6000])

In [ ]:
topics_list = []

for i in range(k):
    # Calculate start and end indices for the current subset
    start_idx = i * 6000
    end_idx = start_idx + 6000
    
    # Extract the subset from the shuffled dataset
    subset = topics[start_idx:end_idx]
    
    topics_list.append(subset)

In [ ]:
print(topics_list[1])

In [10]:
topics_sample = topics.sample(2000, random_state=34)

In [15]:
BM25 = pt.BatchRetrieve(index, wmodel="BM25",
                             controls={"bm25.b" : 0.075, "bm25.k_1": 0.95}
                       ).parallel(4)

In [14]:
BM25_models = []
names = [f"name_{i}" for i in range(7)]
b_values = [0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225]
k1_values = [0.8]

for b in b_values:
    for k1 in k1_values:
        BM25 = pt.BatchRetrieve(index, wmodel="BM25",
                             controls={"bm25.b" : b, "bm25.k_1": k1}
                       ).parallel(4)
        BM25_models.append(BM25)

In [16]:
from pyterrier.measures import *

pt.Experiment(
    retr_systems = [BM25],
    topics = topics,
    qrels = qrels,
    eval_metrics = [NDCG@5, NDCG@10, NDCG@20,
                   RR@5, RR@10, RR@20,
                   P@5, P@10, P@20,
                   R@5, R@10, R@20]
)

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Avoiding reinit of PyTerrier
  warn("Avoiding reinit of PyTerrier")
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Av

13:26:35.259 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
13:26:35.609 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
13:26:35.616 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based mod

,name,nDCG@5,nDCG@10,nDCG@20,RR@5,RR@10,RR@20,P@5,P@10,P@20,R@5,R@10,R@20
0,PoolParallelTransformer(BR(BM25)),0.237043,0.255548,0.281342,0.349581,0.362542,0.368447,0.195927,0.15896,0.126385,0.184259,0.262873,0.364513


In [ ]:
from pyterrier.measures import *

pt.GridSearch(
    BM25,
    params = {BM25: {"bm25.b": [0.45, 0.55, 0.65, 0.85, 0.95, 1.05],
            "bm25.k_1": [0.6, 0.8, 1.0, 1.4, 1.6, 1.8],
    }},
    batch_size = 50,
    topics = topics,
    qrels = qrels,
    metric = NDCG@10
)

In [ ]:
from pyterrier.measures import *

tuned_BM25, _ = pt.KFoldGridSearch(
    BM25,
    params = {BM25: {"bm25.b"  : [0.45, 0.55, 0.65, 0.85, 0.95, 1.05],
            "bm25.k_1": [0.6, 0.8, 1.0, 1.4, 1.6, 1.8]
    }},
    batch_size = 50,
    topics_list = topics_list,
    qrels = qrels,
    metric = NDCG@10
)
#pt.Experiment([pl2, tuned_pl2], all_topics, qrels, ["map"])

In [ ]:
DLM = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"mu": 1500})

In [19]:
DLM_models

[PoolParallelTransformer(BR(/kaggle/working/indexes/stopwords/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'DirichletLM', 'mu': 500},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorate:on', 'querying.allowed.controls': 'scope,qe,qemodel,start,end,site,scope,applypipeline', 'termpipelines': 'St

In [16]:
DLM_models = []
names = [f"name_{i}" for i in range(9)]
mu_values = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500]

for mu in mu_values:
    DLM = pt.BatchRetrieve(index, wmodel="DirichletLM",
                            controls={"mu": mu}
                    ).parallel(4)
    DLM_models.append(DLM)

In [18]:
from pyterrier.measures import *
pt.Experiment(
    retr_systems=DLM_models,
    names = names,
    topics=topics_sample,
    qrels=qrels,
    eval_metrics=[NDCG@10]
)

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


15:58:19.161 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
15:58:19.195 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
15:58:19.292 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based mod

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Avoiding reinit of PyTerrier
  warn("Avoiding reinit of PyTerrier")
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Av

16:33:25.585 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
16:33:25.642 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
16:33:25.665 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based mod

,name,nDCG@10
0,name_0,0.232881
1,name_1,0.232881
2,name_2,0.232881
3,name_3,0.232881
4,name_4,0.232881
5,name_5,0.232881
6,name_6,0.232881
7,name_7,0.232881
8,name_8,0.232881
